# Detyra 2 — Tokenizimi & Embeddings

Qëllimi: të ndërtoni embedding-e për *njësi semantike* (fraza, dialogë, gabime) dhe të matni ngjashmëri.

⚠️ Shënim: Në këtë projekt, dokumentet janë PDF. Për thjeshtësi në kurs, ju mund:
1) të përdorni tekstin e dokumenteve (nëse instruktori ofron .txt/.md paralel), ose
2) të ekstraktoni tekst me PyMuPDF (fitz) / pdfminer.six.

Në këtë notebook ne japim një shembull minimal me PyMuPDF dhe lëmë TODO për pastrim/segmentim.


In [ ]:
# === Setup (run this cell first) ===
import os, json, re, math, random
from pathlib import Path

PROJECT_ROOT = Path("/content")  # In Colab, students will upload the project folder or mount Drive
DOCS_DIR = PROJECT_ROOT / "documents"
QA_PATH = PROJECT_ROOT / "qa" / "qa_benchmark_40.json"

print("Docs dir:", DOCS_DIR)
print("QA path:", QA_PATH)

# Tip: If you're using Google Drive:
# from google.colab import drive
# drive.mount('/content/drive')
# PROJECT_ROOT = Path('/content/drive/MyDrive/<YOUR_PROJECT_FOLDER>')


## 1) Ekstraktim minimal i tekstit nga një PDF
Zgjidhni një PDF dhe ekstraktoni tekstin. Pastaj bëni pastrim bazë.
**TODO:** përmirësoni pastrimin (heqje header/footer, normalizim hapësirash).


In [ ]:
# Install PyMuPDF if needed
!pip -q install pymupdf

import fitz, re
from pathlib import Path

pdf_path = next(iter(sorted(DOCS_DIR.glob("MIL-ENG-001*.pdf"))), None)
print("Using:", pdf_path)

doc = fitz.open(str(pdf_path))
text = ""
for page in doc:
    text += page.get_text() + "\n"
text = re.sub(r"\s+", " ", text).strip()
print("Chars:", len(text))
print(text[:800])


## 2) Segmentim në 'njësi semantike'
Ne synojmë të ndajmë tekstin në njësi sipas pattern-eve si:
- 'FRAZA' / 'TERM' / 'Gabim'
- 'Dialog'

**TODO:** implementoni një segmentues të thjeshtë me regex.


In [ ]:
import re

def segment_units(raw: str):
    # Very simple heuristic splitter; improve it!
    # Splits on headings like 'FRAZA', 'TERM', 'Dialog', 'Gabim'
    pattern = r"(?=\b(FRAZA|TERM|Dialog|Gabim)\b)"
    parts = re.split(pattern, raw)
    # Re-stitch with label
    units = []
    i = 0
    while i < len(parts)-1:
        if parts[i] == "":
            i += 1
            continue
        label = parts[i+1] if parts[i] in ["FRAZA","TERM","Dialog","Gabim"] else None
        if label:
            content = parts[i+2].strip()
            units.append(f"{label} {content}")
            i += 3
        else:
            i += 1
    return [u for u in units if len(u) > 50]

units = segment_units(text)
print("Units:", len(units))
for u in units[:3]:
    print("----")
    print(u[:400])


## 3) Embeddings me SentenceTransformers
Ndërtoni embeddings për 30 njësi dhe llogaritni similarity për disa pyetje.
**TODO:** Shpjegoni pse unit-based embeddings janë më të mira se chunk-e arbitrare.


In [ ]:
!pip -q install sentence-transformers

from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

sample_units = units[:30] if len(units) >= 30 else units
emb = model.encode(sample_units, normalize_embeddings=True)
print("Emb shape:", emb.shape)

query = "Çfarë do të thotë Roger në komunikim radio?"
q_emb = model.encode([query], normalize_embeddings=True)[0]
scores = emb @ q_emb
top = np.argsort(-scores)[:5]
for idx in top:
    print(scores[idx], sample_units[idx][:160])


## Dorëzimi
- Segmentuesi (regex + përmirësime)
- Demonstrim similarity
- Përgjigje: pse ky segmentim është i përshtatshëm për këtë projekt?
